In [10]:
from transformers import AutoConfig, T5ForConditionalGeneration, AutoTokenizer, GenerationConfig
import torch
from unstructured.cleaners.core import clean_extra_whitespace
from pprint import pprint
from underthesea import sent_tokenize


In [2]:
checkpoint = '/home/yuuhanase/FPTU/DAT/vi_news_chatbot/artifacts/new_summarizer_model'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)  
config = AutoConfig.from_pretrained(checkpoint)
model = T5ForConditionalGeneration.from_pretrained(checkpoint, config=config)
model.cuda()

T5ForConditionalGeneration(
  (shared): Embedding(36096, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(36096, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1024, out_features=2816, bias=False)
              (wi_1): Linear(in_features=1024, out_features=2816, bias=False)
       

In [3]:
def clean_text(batch):
    batch = batch.replace("\n", ' ')
    batch = batch.replace("\t", ' ')
    batch = batch.replace("\r", ' ')
    batch = clean_extra_whitespace(batch)
    return batch

def process(text):
    input = clean_text(text)
    tokenized_input = tokenizer(input, return_tensors='pt').to(model.device)
    output = model.generate(
        input_ids=tokenized_input['input_ids'],
        num_beams=1,
        top_k=6,
        top_p=0.9,
        temperature=0.6,
        max_length=129,
        min_length=41,
        num_return_sequences=1,
        repetition_penalty=1.1,
        do_sample=True,
        penalty_alpha=0.6
    )
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

    tokenized_input = tokenized_input.to('cpu')
    output = output.to('cpu')
    del tokenized_input, output
    torch.cuda.empty_cache()
    return decoded_output.replace('. ', '.\n')

In [4]:
from bs4 import BeautifulSoup
import requests
import time

def crawl_content(url):    
    delay = 0.0
    ret = {}
    try:
        while True:
            response = requests.get(url)
            if response.ok:
                break
            time.sleep(0.5 + delay)
            delay += 0.1
            if 0.5 + delay > 1.0:
                print("Time out")
                raise Exception("Time out")
        soup = BeautifulSoup(response.content, "html.parser")
        title = soup.title.text
        article_0 = []
        article_1 = []
        for p in soup.find_all("body")[0].find_all("p"):
            if p.text not in article_0 and not p.find("strong"):
                article_0.append(p.text)
        for p in soup.find_all("p", "Normal"):
            if p.text not in article_1 and not p.find("strong"):
                article_1.append(p.text)
        article = article_0 if len(article_0) >= len(article_1) else article_1
        content = " ".join(article)
        ret['title'] = title
        ret['content'] = content
    except Exception as e:
        print(e)
        ret['title'] = ""
        ret['content'] = ""
    return ret

In [5]:
def pipeline(url):
    news = crawl_content(url)
    news['summary'] = process(news['content'])
    return news

In [13]:
test = []
urls = [
    'https://vnexpress.net/cach-dung-luong-thuong-dieu-khien-lanh-dao-chu-chot-scb-cua-ba-truong-my-lan-4689609.html',
    'https://vnexpress.net/hon-mot-trieu-ty-dong-ba-truong-my-lan-rut-khoi-scb-da-di-dau-4689499.html',
    'https://vnexpress.net/ba-truong-my-lan-bi-truy-to-chiem-doat-hon-304-000-ty-dong-cua-scb-4689256.html',
    'https://vnexpress.net/bo-cong-an-ba-truong-my-lan-rua-tien-vao-bat-dong-san-chuyen-ra-nuoc-ngoai-4694035.html',
    'https://vnexpress.net/bi-an-hon-108-000-ty-dong-tien-mat-ba-truong-my-lan-xuat-khoi-scb-4678936.html'
]
for url in urls:
    test.append(
        pipeline(url)
    )

In [14]:
pprint(test)

[{'content': 'Không chỉ trả lương mỗi tháng 200-500 triệu đồng cho các vị trí '
             'chủ chốt của SCB, bà Trương Mỹ Lan còn thường xuyên cho cổ phần, '
             'thưởng tiền để dễ điều khiển, theo cáo trạng. Trong 45 lãnh đạo, '
             'cán bộ Ngân hàng SCB vừa bị VKSND Tối cao truy tố có 3 cựu Chủ '
             'tịch HĐQT, 4 Tổng giám đốc, 10 Phó tổng giám đốc. Những vị trí '
             'lãnh đạo chủ chốt tại SCB đều là người có trình độ trong lĩnh '
             'vực tài chính, ngân hàng, được bà Trương Mỹ Lan, Chủ tịch Tập '
             'đoàn Vạn Thịnh Phát (có quyền lực cao nhất tại SCB vì chi phối '
             'hơn 91% cổ phần) tuyển chọn. Theo nhà chức trách, bà Lan đã trả '
             'lương 200-500 triệu đồng mỗi tháng cho họ và tặng thưởng tiền, '
             'cổ phần SCB để "thu phục". Những cánh tay đắc lực như cựu chủ '
             'tịch HĐQT Nguyễn Thị Thu Sương, Đinh Văn Thành, Bùi Anh Dũng và '
             'cựu tổng, phó tổng giám đốc Võ Tấn

In [18]:
big_news = ''
for data in test:
    big_news += data['summary'] + '\n\n'
print(big_news)
ret = process(big_news)
print("\n".join(sent_tokenize(ret)))

Bà Trương Mỹ Lan, Chủ tịch Tập đoàn Vạn Thịnh Phát và 10 Phó tổng giám đốc của Ngân hàng SCB bị VKSND Tối cao truy tố về tội Đưa hối lộ, Vi phạm quy định về hoạt động của ngân hàng, Tham ô tài sản, Thiếu trách nhiệm gây hậu quả nghiêm trọng.
Họ đã giúp bà Lan rút ruột của ngân hàng hơn 1.066.600 tỷ đồng trong 10 năm.
Bà Lan cùng các cán bộ khác của ngân hàng đã giúp họ rút tiền của ngân hàng hơn 1.066.60 tỷ đồng trong 10 năm.

Bà Trương Mỹ Lan, Chủ tịch Tập đoàn Vạn Thịnh Phát, bị VKSND Tối cao truy tố về ba tội Đưa hối lộ, Vi phạm quy định về hoạt động của ngân hàng và Tham ô tài sản.
Theo cáo trạng, bà Lan đã chuyển phần lớn tiền ra khỏi ngân hàng sau khi được giải ngân rồi dùng nhiều thủ đoạn để "cắt đứt dòng tiền".
Bà Lan còn bị cáo buộc chiếm đoạt 304.000 tỷ đồng và gây thiệt hại hơn 64.600 tỷ đồng.
80 người còn lại đều thành khẩn khai báo, ăn năn hối cải.

VKSND Tối cao đã truy tố 85 bị can về các tội: Tham ô tài sản, Nhận hối lộ, Vi phạm quy định về hoạt động của ngân hàng và Th

Bà Trương Mỹ Lan, Chủ tịch Tập đoàn Vạn Thịnh Phát và 10 Phó tổng giám đốc của Ngân hàng SCB bị VKSND Tối cao truy tố về các tội: Tham ô tài sản, Nhận hối lộ, Vi phạm quy định về hoạt động của ngân hàng và Tham ô tài sản.
Bà Lan bị cáo buộc chiếm đoạt hơn 304.000 tỷ đồng và gây thiệt hại hơn 64.600 tỷ đồng.
VKSND Tối cao đã đề nghị truy tố 86 bị can về 6 tội danh.


In [19]:
big_news = ''
for data in test:
    big_news += data['content'] + '\n\n'
print(big_news)
ret = process(big_news)
print("\n".join(sent_tokenize(ret)))

Không chỉ trả lương mỗi tháng 200-500 triệu đồng cho các vị trí chủ chốt của SCB, bà Trương Mỹ Lan còn thường xuyên cho cổ phần, thưởng tiền để dễ điều khiển, theo cáo trạng. Trong 45 lãnh đạo, cán bộ Ngân hàng SCB vừa bị VKSND Tối cao truy tố có 3 cựu Chủ tịch HĐQT, 4 Tổng giám đốc, 10 Phó tổng giám đốc. Những vị trí lãnh đạo chủ chốt tại SCB đều là người có trình độ trong lĩnh vực tài chính, ngân hàng, được bà Trương Mỹ Lan, Chủ tịch Tập đoàn Vạn Thịnh Phát (có quyền lực cao nhất tại SCB vì chi phối hơn 91% cổ phần) tuyển chọn. Theo nhà chức trách, bà Lan đã trả lương 200-500 triệu đồng mỗi tháng cho họ và tặng thưởng tiền, cổ phần SCB để "thu phục". Những cánh tay đắc lực như cựu chủ tịch HĐQT Nguyễn Thị Thu Sương, Đinh Văn Thành, Bùi Anh Dũng và cựu tổng, phó tổng giám đốc Võ Tấn Hoàng Văn, Trương Khánh Hoàng, Trần Thị Mỹ Dung đã giúp bà Lan rút ruột của SCB hơn 1.066.600 tỷ đồng trong 10 năm. Bà Lan cùng các cán bộ trên của SCB vừa bị VKS truy tố về tội Đưa hối lộ, Vi phạm quy địn